## Find Artist Genres
#### Import Spotipy module

In [1]:
# find artist genres
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy
import os
from dotenv import load_dotenv

#### Store Credentials in .env file

In [2]:
# enter spotify credentials
load_dotenv()
client_id = os.getenv("SPOTIFY_CLIENT_ID")
client_secret = os.getenv("SPOTIFY_CLIENT_SECRET")

#### Access Spotify API

In [3]:
# authenticate with Spotify
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)


SpotifyOauthError: No client_id. Pass it or set a SPOTIPY_CLIENT_ID environment variable.

#### Loop Through Artist List and Add Genre to List

In [ ]:
# read artist csv
input_path = './Cleaned_Data/Artist_List.csv'
artist_df = pd.read_csv(input_path)
artist_names = artist_df['artist_name'].tolist()

# create a list to store results
artist_genre =[]

# find genres for each artist
for artist_name in artist_names:
    try:
        # search for artist on Spotify
        results = sp.search(q=artist_name,type='artist', limit =1)
        if results['artists']['items']:
            artist = results['artists']['items'][0]
            genres = ', '.join(artist['genres']) if artist['genres'] else 'No genres found'
        else:
            genres = 'Not Found'
    except Exception as e:
        genres = f'Error: {e}'
    artist_genre.append(genres)

# create DataFrame with the results
artist_genre_data = pd.DataFrame({
    'artist_name': artist_names,
    'genres': artist_genre
})

#### Normalize Genre Labels

In [ ]:
# clean the genres
# normalize genres
artist_genre_data['genres'] = artist_genre_data['genres'].str.lower()
artist_genre_data['genres'] = artist_genre_data['genres'].str.strip()
# define genre mapping
# used regular expression \b (word boundaries) for precise genre replacement to avoid partial matches 
genre_mapping = {
    r'\bc-pop\b': 'mandopop',
    r'\bclassic mandopop\b': 'mandopop',
    r'\bmando pop\b': 'mandopop',
    r'\bmandarin pop\b': 'mandopop',
    r'\bchinese pop\b': 'mandopop',
    r'\btaiwanese pop\b': 'mandopop',
    r'\bmainland chinese pop\b': 'mandopop',
    r'\btaiwan pop\b': 'mandopop',
    r'\bchinese jazz\b': 'jazz',
    r'\bpop dance\b': 'dance pop'
}


# replace substrings in genres
for old_genre, new_genre in genre_mapping.items():
    artist_genre_data['genres'] = artist_genre_data['genres'].str.replace(old_genre,new_genre,regex=True) #regex=True ensures \b to function

# remove duplicates in genres
artist_genre_data['genres'] = artist_genre_data['genres'].str.split(', ').apply(lambda x: ', '.join(sorted(set(x))))

artist_df['genre'] = artist_genre

#### Create new CSV and Save

In [ ]:
# save to csv
updated_path = './Cleaned_Data/Artist_Genre_List.csv'
artist_df.to_csv(updated_path, index= False, encoding= 'utf-8-sig')